In [ ]:
# PanNuke

In [ ]:
import numpy as np
from PIL import Image
import os, json, glob, shutil, random, cv2
import matplotlib.pyplot as plt
import skimage
from skimage.measure import regionprops, find_contours, label
from pycocotools import mask
from pycocotools.coco import COCO
import matplotlib.pyplot as plt

In [ ]:
# PanNuke dataset dir
basedir = 'PanNuke'

In [ ]:
MAIN_CLASSES = ('Neoplastic cells', 'Inflammatory', 'Connective/Soft tissue cells', 'Dead Cells', 'Epithelial')
main_cls_arr = []
for i in range(len(MAIN_CLASSES)):
    cls_dict = {'id': i, 'name':MAIN_CLASSES[i]}
    main_cls_arr.append(cls_dict)

In [ ]:
# Prepare rgb image
for fold in range(3):
    imgdir = f'{basedir}/images/fold{fold+1}'
    img_data = np.load(f'{imgdir}/images.npy')
    for i in range(img_data.shape[0]):
        img = Image.fromarray(img_data[i].astype(np.uint8))
        os.makedirs(f'{imgdir}/rgb', exist_ok=True)
        if not os.path.exists(f'{imgdir}/rgb/{i+1}.png'):
            img.convert('RGB').save(f'{imgdir}/rgb/{i+1}.png')

In [ ]:
# Prepare rgb_seg image
for fold in range(3):
    imgdir = f'{basedir}/masks/fold{fold+1}'
    img_data = np.load(f'{imgdir}/masks.npy')
    for i in range(img_data.shape[0]):
        img = 1 - img_data[i, :, :, 5]
        img = Image.fromarray(img.astype(np.uint8))
        os.makedirs(f'{imgdir}/rgb_seg', exist_ok=True)
        if not os.path.exists(f'{imgdir}/rgb_seg/{i+1}.png'):
            img.save(f'{imgdir}/rgb_seg/{i+1}.png')

In [ ]:
def annt_info(basedir, fold, keep_crowd=True):
    imgdir = f'{basedir}/images/fold{fold+1}'
    maskdir = f'{basedir}/masks/fold{fold+1}'
    mask_data = np.load(f'{maskdir}/masks.npy')
    type_data = np.load(f'{imgdir}/types.npy')

    img_li = []
    annt_li = []

    annt_id = 1
    for idx in range(mask_data.shape[0]):
        mask_i = mask_data[idx, :, :, :-1]
        h, w, _ = mask_i.shape
        img_dict = {
                    'fold':fold+1,
                    'file_name': f'{idx+1}.png',
                    'height': h,
                    'width': w,
                    'id': idx+1,
                    'type': type_data[idx],
                    'n_objects': len(np.unique(mask_i))-1
                   }
        img_li.append(img_dict)
        
        for catg_id in range(5):
            tmp_mask = mask_i[:, :, catg_id]
            annt_id_li = np.delete(np.unique(tmp_mask), 0)
            if len(annt_id_li) != 0:
                for inst_id in annt_id_li:
                    tmp_inst_mask = np.array(tmp_mask == inst_id, dtype=np.uint8)
                    RLE_inst = mask.encode(np.asfortranarray(tmp_inst_mask))
                    RLE_inst['counts'] = RLE_inst['counts'].decode('ascii')
                    bbox = mask.toBbox(RLE_inst).tolist()
                    # bbox:[x, y, w, h]
                    area = bbox[2]*bbox[3]
                    segmentation = RLE_inst
                    iscrowd = 0

                    annt_dict = {
                        'bbox': bbox,
                        'area':area,
                        'image_id': idx+1,
                        'category_id':catg_id,
                        'id':int(annt_id*3+fold),
                        'iscrowd':iscrowd,
                        'segmentation':segmentation, 
                    }
                    annt_id+=1
                    if not keep_crowd and iscrowd == 1:
                        continue
                    else:
                        annt_li.append(annt_dict)
    print(f'annt id num:{annt_id}')
    return img_li, annt_li

In [ ]:
for fold in range(3):
    print(f'Preprocessing fold:{fold+1}')
    img_li, annt_li = annt_info(basedir, fold, keep_crowd=True)
    fold_annt_json = {
    'images': img_li,
    'annotations': annt_li,
    'categories': main_cls_arr
    }
    with open(f'{basedir}/PanNuke_annt_RLE_fold{fold+1}.json', 'w') as f:
        json.dump(fold_annt_json, f)